## Import Modules

In [1]:
import os
os.chdir('/Users/Evan/DataScience/TB_GM')
import numpy as np
import pandas as pd
from GM11 import GM11
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

In [6]:
%ls

Functions/              Papers/                 gdp.csv
GM11.py*                README.md               grey_model.R
Grey_model_2025.ipynb   Trash/                  nation.csv
Grey_model_TB.ipynb     __pycache__/
National TB Statistics/ data/


## Import data

In [2]:
inputfile = './data/nation_TB1.csv'
nation = pd.read_csv(inputfile)[['incidence','death','incidence rate','death rate']]
nation

,incidence,death,incidence rate,death rate
0,1259309,3402,96.878875,0.261716
1,1127571,3339,86.234749,0.255361
2,1163959,3669,88.549250,0.279123
3,1169540,2802,88.515012,0.212065
4,1076938,3783,81.093508,0.284860
5,991350,3000,74.272892,0.224763
6,953275,2840,71.091647,0.211796
7,951508,2662,70.620700,0.197573
8,904434,2576,66.795220,0.190245
9,889381,2240,65.628808,0.165293


In [3]:
nation.index=range(2005,2015)
nation

,incidence,death,incidence rate,death rate
2005,1259309,3402,96.878875,0.261716
2006,1127571,3339,86.234749,0.255361
2007,1163959,3669,88.549250,0.279123
2008,1169540,2802,88.515012,0.212065
2009,1076938,3783,81.093508,0.284860
2010,991350,3000,74.272892,0.224763
2011,953275,2840,71.091647,0.211796
2012,951508,2662,70.620700,0.197573
2013,904434,2576,66.795220,0.190245
2014,889381,2240,65.628808,0.165293


In [4]:
nation.to_csv('nation.csv')

In [5]:
for k in range(2015,2026):
    nation.loc[k] = None

## GM(1,1)

In [ ]:
nation['incidence_rate'] = None
nation['death_rate'] = None
for k in range(2005,2026):
    f = GM11(nation['incidence rate'].loc[range(2005,2015)].as_matrix())
    g = GM11(nation['death rate'].loc[range(2005,2015)].as_matrix())
    nation['incidence_rate'].loc[k] = f[0](k-2005+1).round(2)
    nation['death_rate'].loc[k] = g[0](k-2005+1).round(2)
nation

In [ ]:
print('后残差比:',f[4],';','小残差率',f[5])

In [ ]:
# nation['death_rate'] = None
# for k in range(2005,2026):
#     f = GM11(nation['death rate'].loc[range(2005,2015)].as_matrix())
#     nation['death_rate'].loc[k] = f[0](k-2005+1).round(2)
# nation

In [ ]:
print('后残差比:',f[4],';','小残差率',f[5])

## 滑动平均计算

In [ ]:
nation['death.rate'] = None
for i in range(2005,2015):
    if i == 2005:
        nation['death.rate'].loc[2005] = (nation['death rate'].loc[2005]*3+nation['death rate'].loc[2006])/4
    elif i == 2014:
        nation['death.rate'].loc[2014] = (nation['death rate'].loc[2013]+nation['death rate'].loc[2014]*3)/4
    else:
        nation['death.rate'].loc[i] = (nation['death rate'].loc[i-1]+2*nation['death rate'].loc[i]+nation['death rate'].loc[i+1])/4

In [ ]:
mv = pd.Series([0.260128, 0.26289, 0.256418, 0.247028, 0.251637, 0.236546, 0.211482, 0.199297, 0.185839, 0.171531],index=range(2005,2015))
nation['death.rate'] = mv

In [ ]:
nation['death_rate_mv'] = None
for k in range(2005,2026):
    g = GM11(nation['death.rate'].loc[range(2005,2015)].as_matrix())
    nation['death_rate_mv'].loc[k] = g[0](k-2005+1).round(2)
nation

In [ ]:
print('后残差比:',g[4],';','小残差率',g[5])

In [ ]:
from bokeh.plotting import figure, output_notebook,show
from bokeh.layouts import gridplot
output_notebook()
p = figure(plot_width=600,plot_height=400,title='Predicton with Grey Model',tools='')
p.grid.grid_line_alpha=0.3
p.xaxis.axis_label ='Year'
p.yaxis.axis_label='Incidence Rate'
p.line(range(2005,2026),nation['incidence_rate'],line_dash="4 4")
p.line(range(2005,2026),nation.loc[2005:2014,'incidence rate'],line_color='red')
p.axis.minor_tick_in = 0
p.axis.minor_tick_out = 0
show(p)